In [4]:
import pandas as pd
import numpy as np
import os
import re
merchant_df = pd.read_parquet('../data/curated/final_merchant_statistics')

In [6]:
merchant_df.columns

Index(['merchant_abn', 'name', 'tags', 'tag', 'revenue_level', 'take_rate',
       'sales_revenue', 'discounted_sales_revenue', 'num_orders',
       'approximate_fraudulent_orders', 'avg_daily_rev',
       'discounted_avg_daily_rev', 'avg_value_per_order',
       'discounted_avg_value_per_order', 'avg_daily_orders',
       'avg_daily_approximate_fraudulent_orders', 'std_daily_revenue',
       'std_daily_discounted_revenue', 'avg_daily_commission',
       'discounted_avg_daily_commission', 'avg_commission_per_order',
       'discounted_avg_commission_per_order', 'overall_commission',
       'discounted_overall_commission', 'overall_fraud_rate',
       'sa2_region_count', 'median_customer_income', 'returning_customers',
       'unique_customers', 'mean_spending', 'std_spending', 'vip_customers'],
      dtype='object')

In [8]:
merchant_df.head()

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,overall_commission,discounted_overall_commission,overall_fraud_rate,sa2_region_count,median_customer_income,returning_customers,unique_customers,mean_spending,std_spending,vip_customers
0,10023283211,Felis Limited,"((furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",e,0.18,7.032777e+05,6.554602e+05,3261,216.117177,...,1265.899881,1179.828379,0.066273,1717.0,794.816023,11.0,3032.0,231.951752,155.424439,137.0
1,10346855916,Odio Institute,"((equipment, tool, furniture, and appliance r...","equipment, tool, furniture, and appliance ren...",b,3.57,6.775178e+03,6.166115e+03,6,0.492377,...,241.873872,220.130319,0.082063,7.0,788.166667,0.0,6.0,1129.196413,717.988423,0.0
2,10385163239,Sed Et Company,"([florists supplies, nursery stock, and flower...","florists supplies, nursery stock, and flowers",a,6.61,2.519346e+04,2.342538e+04,74,5.026389,...,1665.287911,1548.417412,0.067924,132.0,797.507171,0.0,73.0,345.115933,203.496326,3.0
3,10648956813,Proin Nisl Institute,"[(computeRs, computer peripheral equipment, an...","computers, computer peripheral equipment, and ...",a,6.66,1.419600e+06,1.329626e+06,21981,1390.109580,...,94545.379790,88553.112413,0.063241,2208.0,825.551750,1574.0,14380.0,98.720466,57.083817,710.0
4,10714068705,Sollicitudin Commodo Ipsum Industries,"([furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",c,2.51,4.355100e+05,4.067113e+05,3438,221.458950,...,10931.299762,10208.454670,0.064415,1715.0,796.782831,10.0,3221.0,135.209547,113.737044,145.0


In [ ]:
Exposure:
-  'num_orders'
- 'sa2_region_count'
- 'returning_customers'
- 'unique_customers'

Risk of Default Payment
- 'median_customer_income'
- 'vip_customers'

Fraud
- 'discounted_sales_revenue'
- 'discounted_avg_daily_rev'
- 'approximate_fraudulent_orders'
- 'discounted_overall_commission'

Sales Performance
- 'avg_daily_rev'
- 'avg_daily_orders'
- 'avg_daily_commission'
- 'std_spending' 

     'take_rate',
       'sales_revenue', 'discounted_sales_revenue', 'num_orders',
       'approximate_fraudulent_orders', 'avg_daily_rev',
       'discounted_avg_daily_rev', 'avg_value_per_order',
       'discounted_avg_value_per_order', 'avg_daily_orders',
       'avg_daily_approximate_fraudulent_orders', 'std_daily_revenue',
       'std_daily_discounted_revenue', 'avg_daily_commission',
       'discounted_avg_daily_commission', 'avg_commission_per_order',
       'discounted_avg_commission_per_order', 'overall_commission',
       'discounted_overall_commission', 'overall_fraud_rate',
       'sa2_region_count', 'median_customer_income', 'returning_customers',
       'unique_customers', 'mean_spending', 'std_spending', 'vip_customers'

In [26]:
risk=['std_daily_revenue','discounted_sales_revenue',
       'approximate_fraudulent_orders','avg_daily_approximate_fraudulent_orders',
       'discounted_avg_value_per_order','std_daily_discounted_revenue',
       'discounted_avg_daily_rev','discounted_avg_daily_commission',
       'discounted_avg_commission_per_order','overall_fraud_rate',
       'discounted_overall_commission' ]


consumer_quality=['median_customer_income', 'returning_customers']
revenue=['sales_revenue','avg_daily_rev','take_rate','avg_daily_commission',
         'overall_commission','avg_commission_per_order','avg_value_per_order','mean_spending','std_spending']


exposure=['avg_daily_orders','sa2_region_count','num_orders','unique_customers','vip_customers']

## PCA for ranking

In [32]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

merchant_df = merchant_df.dropna()

#



features = risk+consumer_quality+revenue+exposure

# some features the lower the better
EPSILON=0.000001
merchant_df["approximate_fraudulent_orders"]=1/(merchant_df["approximate_fraudulent_orders"]+EPSILON)
merchant_df["avg_daily_approximate_fraudulent_orders"]=1/(merchant_df["avg_daily_approximate_fraudulent_orders"]+EPSILON)
merchant_df["overall_fraud_rate"]=1/(merchant_df["overall_fraud_rate"]+EPSILON)

merchant_df["std_daily_revenue"]=1/(merchant_df["std_daily_revenue"]+EPSILON)
merchant_df["std_daily_discounted_revenue"]=1/(merchant_df["std_daily_discounted_revenue"]+EPSILON)
merchant_df["std_spending"]=1/(merchant_df["std_spending"]+EPSILON)

x = merchant_df.loc[:, features].values
x = StandardScaler().fit_transform(x)

## we have 4 dimensions , PCA component is 4

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

resultDF = pd.concat([principalDf,merchant_df["merchant_abn"]], axis = 1)

In [33]:
resultDF.head()

,principal component 1,principal component 2,principal component 3,merchant_abn
0,0.512001,-1.914379,-0.522354,1.002328e+10
1,-2.448405,3.920628,1.044115,1.034686e+10
2,-1.560276,-0.484241,0.279504,1.038516e+10
3,5.873537,-2.244544,0.548649,1.064896e+10
4,0.394594,-2.323943,-0.374619,1.071407e+10


In [34]:
weights = pca.components_
weights

array([[ 0.2164086 ,  0.25439836,  0.22303113,  0.22141446, -0.07439101,
         0.21783049,  0.25439836,  0.24934859, -0.06047235, -0.07414378,
         0.24934859,  0.00420954,  0.20678106,  0.25395368,  0.25395368,
         0.01753087,  0.24900694,  0.24900694, -0.06059476, -0.07434322,
        -0.05424147, -0.03680086,  0.21866352,  0.17759977,  0.21866352,
         0.23050638,  0.22045521],
       [ 0.14552176,  0.07527598, -0.00940564, -0.0085501 ,  0.36132399,
         0.1432954 ,  0.07527598,  0.07878154,  0.34035378,  0.36020906,
         0.07878154, -0.00846358,  0.01146007,  0.07625468,  0.07625468,
         0.03640481,  0.0797781 ,  0.0797781 ,  0.34057784,  0.36104417,
         0.36847088,  0.34335089, -0.01055223, -0.14673222, -0.01055223,
        -0.0713459 , -0.07415526],
       [-0.24329292, -0.13258375,  0.3610064 ,  0.36531642,  0.07438215,
        -0.24155907, -0.13258375, -0.12023132,  0.05142432,  0.07493915,
        -0.12023132, -0.01417554,  0.38109732, -0.1356

In [35]:
pca.explained_variance_ratio_


array([0.50545361, 0.24703531, 0.08858611])

In [37]:
print(sum(pca.explained_variance_ratio_))

0.8410750302356046


## Feature selection by pca weights

In [48]:
from operator import add

absolute_weights =[0]*27
for component_weight in weights:
    absolute_weights = list(map(add, absolute_weights, np.abs(component_weight)))

index_weight_map = []
for i in range(len(absolute_weights)):
    index_weight_map.append((absolute_weights[i],i))

index_weight_map.sort(key=lambda tup: tup[0], reverse=True)

for i in range(15):
    print(features[index_weight_map[i][1]])

std_daily_revenue
avg_daily_orders
num_orders
std_daily_discounted_revenue
returning_customers
avg_daily_approximate_fraudulent_orders
approximate_fraudulent_orders
avg_value_per_order
discounted_avg_value_per_order
overall_fraud_rate
mean_spending
sa2_region_count
avg_daily_rev
sales_revenue
discounted_sales_revenue
